In [2]:
import pymongo
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Connect to MongoDB
client = pymongo.MongoClient("mongodb+srv://admin:<password>@cluster0.jwzyj.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")
db = client["dining_info"]
collection = db["dining_data"]

# Fetch all records
data = list(collection.find())

# Convert MongoDB data to DataFrame
df = pd.DataFrame(data)

# Data Cleaning and Transformation
if "_id" in df.columns:
    df.drop("_id", axis=1, inplace=True)

df["order_time"] = pd.to_datetime(df["order_time"], errors="coerce")
df["price_for_1"] = pd.to_numeric(df["price_for_1"], errors="coerce")
df["Qty"] = pd.to_numeric(df["Qty"], errors="coerce")
df["age"] = pd.to_numeric(df["age"], errors="coerce")

df.fillna({"price_for_1": df["price_for_1"].median(), "age": df["age"].median(), "Qty": 1}, inplace=True)

# Add revenue column
df["total_revenue"] = df["price_for_1"] * df["Qty"]

# Cuisine options
cuisine_options = df["Preferred Cusine"].dropna().unique()

# Initialize Dash App
app = dash.Dash(__name__)

def filter_data(start_date, end_date, selected_cuisines):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    filtered_df = df[(df["order_time"].dt.date >= start_date.date()) & (df["order_time"].dt.date <= end_date.date())]

    if selected_cuisines:
        filtered_df = filtered_df[filtered_df["Preferred Cusine"].isin(selected_cuisines)]
    
    return filtered_df

# Generate charts
def generate_pie_chart(filtered_df):
    avg_cost_by_cuisine = filtered_df.groupby("Preferred Cusine")["price_for_1"].mean().reset_index()
    return px.pie(avg_cost_by_cuisine, values="price_for_1", names="Preferred Cusine", title="Average Dining Cost by Cuisine")

def generate_line_chart(filtered_df):
    filtered_df["order_date"] = filtered_df["order_time"].dt.date
    customer_count = filtered_df.groupby("order_date")["customer_id"].nunique().reset_index()
    return px.line(customer_count, x="order_date", y="customer_id", title="Customer Count Over Time")

def generate_revenue_chart(filtered_df):
    revenue_by_cuisine = filtered_df.groupby("Preferred Cusine")["total_revenue"].sum().reset_index()
    return px.bar(revenue_by_cuisine, x="Preferred Cusine", y="total_revenue", title="Total Revenue by Cuisine")

def generate_avg_age_chart(filtered_df):
    avg_age_by_cuisine = filtered_df.groupby("Preferred Cusine")["age"].mean().reset_index()
    return px.bar(avg_age_by_cuisine, x="Preferred Cusine", y="age", title="Average Age of Customers by Cuisine")

def generate_revenue_trend(filtered_df):
    filtered_df["order_date"] = filtered_df["order_time"].dt.date
    revenue_trend = filtered_df.groupby("order_date")["total_revenue"].sum().reset_index()
    return px.line(revenue_trend, x="order_date", y="total_revenue", title="Daily Revenue Trend")

# App Layout
app.layout = html.Div(children=[
    html.H1("Dining Insights Dashboard", style={"text-align": "center"}),

    html.Div([
        html.H3("Select Date Range"),
        dcc.DatePickerRange(
            id="date-picker-range",
            start_date=df["order_time"].min().date(),
            end_date=df["order_time"].max().date(),
            display_format="YYYY-MM-DD",
        ),
    ], style={"margin-bottom": "20px"}),

    html.Div([
        html.H3("Select Preferred Cuisine"),
        dcc.Dropdown(
            id="cuisine-dropdown",
            options=[{"label": cuisine, "value": cuisine} for cuisine in cuisine_options],
            multi=True,
            placeholder="Select cuisines",
        ),
    ], style={"margin-bottom": "20px"}),

    dcc.Graph(id="pie-chart"),
    dcc.Graph(id="line-chart"),
    dcc.Graph(id="revenue-chart"),
    dcc.Graph(id="avg-age-chart"),
    dcc.Graph(id="revenue-trend")
])

# Callbacks
@app.callback(
    [Output("pie-chart", "figure"),
     Output("line-chart", "figure"),
     Output("revenue-chart", "figure"),
     Output("avg-age-chart", "figure"),
     Output("revenue-trend", "figure")],
    [Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date"),
     Input("cuisine-dropdown", "value")]
)
def update_charts(start_date, end_date, selected_cuisines):
    filtered_df = filter_data(start_date, end_date, selected_cuisines)

    pie_chart = generate_pie_chart(filtered_df)
    line_chart = generate_line_chart(filtered_df)
    revenue_chart = generate_revenue_chart(filtered_df)
    avg_age_chart = generate_avg_age_chart(filtered_df)
    revenue_trend = generate_revenue_trend(filtered_df)

    return pie_chart, line_chart, revenue_chart, avg_age_chart, revenue_trend

# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)


C:\Users\supri\AppData\Local\Temp\ipykernel_21544\360963858.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\supri\AppData\Local\Temp\ipykernel_21544\360963858.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

